In [24]:
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load the tables 
covid_df = pd.read_csv('../data/covid.csv', index_col=False)
precovid_df = pd.read_csv('../data/precovid.csv', index_col=False)
postcovid_df = pd.read_csv('../data/postcovid.csv', index_col=False)

In [25]:
def get_embedded_report(inp_df, estimator, count):
    
    input_df = inp_df.drop(columns={'Number of Workers','percent_layoff','Unnamed: 0'})
    X = input_df
    y = inp_df['Number of Workers']
    
    # First normalize our predictors
    X_numeric_scaled = StandardScaler().fit_transform(X)
    y_scaled     = StandardScaler().fit_transform(y.values.reshape(-1,1))

    # Save the columns name into variable named 'features'
    features = X.columns

    model = estimator.fit(X, y_scaled)
    try:
        c = model.coef_
    except:
        c = model.feature_importances_
    finally:
        output_df = pd.DataFrame(list(zip(features,c)), columns=['Features','Coefficient'])
        return output_df[output_df['Coefficient']>0].sort_values(by=['Coefficient'], ascending=False).reset_index(drop=True).head(count)

In [26]:
count = 20
precovid_features = get_embedded_report(precovid_df, Lasso(alpha=0.1), count)
covid_features = get_embedded_report(precovid_df, Lasso(alpha=0.1), count)
postcovid_features = get_embedded_report(precovid_df, Lasso(alpha=0.1), count)

/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.184e+02, tolerance: 3.430e-02
  model = cd_fast.enet_coordinate_descent(
/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.184e+02, tolerance: 3.430e-02
  model = cd_fast.enet_coordinate_descent(
/Users/Bhavya/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

In [27]:
print(precovid_features)
precovid_df[precovid_features['Features']]

                             Features   Coefficient
0                   Number of Layoffs  5.438326e-02
1                   industry_labelled  6.643478e-04
2                      employee_count  4.274054e-06
3                  capitalExpenditure  8.818919e-10
4                   commonStockIssued  7.899566e-10
5      ResearchAndDevelopmentExpenses  6.368232e-10
6                           netIncome  5.196202e-10
7                         grossProfit  4.266340e-10
8         depreciationAndAmortization  3.264021e-10
9          effectOfForexChangesOnCash  3.228086e-10
10                      otherExpenses  2.354769e-10
11        SellingAndMarketingExpenses  1.663071e-10
12      inventory_cash-flow-statement  1.558306e-10
13                   incomeTaxExpense  1.503780e-10
14              weightedAverageShsOut  1.498765e-10
15             commonStockRepurchased  8.939251e-11
16   GeneralAndAdministrativeExpenses  7.370204e-11
17          deferredRevenueNonCurrent  6.703442e-11
18          

,Number of Layoffs,industry_labelled,employee_count,capitalExpenditure,commonStockIssued,ResearchAndDevelopmentExpenses,netIncome,grossProfit,depreciationAndAmortization,effectOfForexChangesOnCash,otherExpenses,SellingAndMarketingExpenses,inventory_cash-flow-statement,incomeTaxExpense,weightedAverageShsOut,commonStockRepurchased,GeneralAndAdministrativeExpenses,deferredRevenueNonCurrent,freeCashFlow,deferrredTaxLiabilitiesNonCurrent
0,1,2,118033.0,-1.018000e+09,0.0,0.000000e+00,6.620000e+08,2.946000e+09,0.000000e+00,0.0,1.271000e+09,401000000.0,0.000000e+00,2.200000e+08,4.450080e+08,-1.700000e+07,0.0,5.249000e+09,-2.820000e+08,0.0
1,1,4,132000.0,-3.267000e+09,1000000.0,3.701000e+09,1.151900e+10,2.042100e+10,2.665000e+09,0.0,0.000000e+00,0.0,1.693000e+09,1.765000e+09,1.952867e+10,-2.078300e+10,0.0,2.878000e+09,1.122100e+10,398000000.0
2,1,4,137000.0,-2.363000e+09,0.0,3.948000e+09,1.156100e+10,2.182100e+10,3.040000e+09,0.0,0.000000e+00,0.0,7.000000e+07,2.232000e+09,1.869628e+10,-2.370200e+10,0.0,0.000000e+00,8.792000e+09,0.0
3,1,5,3641.0,0.000000e+00,0.0,0.000000e+00,5.988000e+07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,6.879000e+06,9.541878e+07,0.000000e+00,0.0,0.000000e+00,6.597500e+07,0.0
4,1,11,6700.0,-8.000000e+06,0.0,0.000000e+00,1.040000e+07,1.275000e+08,1.830000e+07,400000.0,9.300000e+06,0.0,-4.350000e+07,-4.300000e+06,1.068000e+08,-9.100000e+06,0.0,0.000000e+00,5.240000e+07,174800000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,1,568,7700.0,-2.650000e+07,0.0,1.010000e+07,5.520000e+07,1.356000e+08,2.620000e+07,-200000.0,-2.700000e+06,0.0,0.000000e+00,8.000000e+06,7.390000e+07,0.000000e+00,64900000.0,0.000000e+00,6.190000e+07,11700000.0
339,2,570,11500.0,-1.850000e+07,0.0,0.000000e+00,1.350000e+08,5.383000e+08,6.110000e+07,0.0,0.000000e+00,0.0,0.000000e+00,2.720000e+07,4.233000e+08,-1.422000e+08,0.0,0.000000e+00,2.442000e+08,161300000.0
340,1,570,11500.0,-3.420000e+07,0.0,0.000000e+00,1.354000e+08,5.512000e+08,6.700000e+07,0.0,0.000000e+00,0.0,0.000000e+00,6.210000e+07,4.195000e+08,-6.880000e+07,0.0,0.000000e+00,2.151000e+08,152100000.0
341,1,571,9000.0,-2.356400e+07,9565000.0,4.066100e+07,6.610700e+07,1.894890e+08,0.000000e+00,1081000.0,0.000000e+00,0.0,0.000000e+00,2.620700e+07,6.194100e+07,-7.126200e+07,52980000.0,3.329270e+08,5.468400e+07,173289000.0


In [28]:
print(covid_features)
covid_df[covid_features['Features']]


                             Features   Coefficient
0                   Number of Layoffs  5.438326e-02
1                   industry_labelled  6.643478e-04
2                      employee_count  4.274054e-06
3                  capitalExpenditure  8.818919e-10
4                   commonStockIssued  7.899566e-10
5      ResearchAndDevelopmentExpenses  6.368232e-10
6                           netIncome  5.196202e-10
7                         grossProfit  4.266340e-10
8         depreciationAndAmortization  3.264021e-10
9          effectOfForexChangesOnCash  3.228086e-10
10                      otherExpenses  2.354769e-10
11        SellingAndMarketingExpenses  1.663071e-10
12      inventory_cash-flow-statement  1.558306e-10
13                   incomeTaxExpense  1.503780e-10
14              weightedAverageShsOut  1.498765e-10
15             commonStockRepurchased  8.939251e-11
16   GeneralAndAdministrativeExpenses  7.370204e-11
17          deferredRevenueNonCurrent  6.703442e-11
18          

,Number of Layoffs,industry_labelled,employee_count,capitalExpenditure,commonStockIssued,ResearchAndDevelopmentExpenses,netIncome,grossProfit,depreciationAndAmortization,effectOfForexChangesOnCash,otherExpenses,SellingAndMarketingExpenses,inventory_cash-flow-statement,incomeTaxExpense,weightedAverageShsOut,commonStockRepurchased,GeneralAndAdministrativeExpenses,deferredRevenueNonCurrent,freeCashFlow,deferrredTaxLiabilitiesNonCurrent
0,2,2,118033.0,-5.770000e+08,0.0,0.000000e+00,-2.399000e+09,-2.437000e+09,0.000000e+00,0.0,6.590000e+08,70000000.0,0.0,-6.960000e+08,5.090490e+08,0.0,0.000000e+00,7.043000e+09,-3.181000e+09,0.000000e+00
1,3,2,118033.0,0.000000e+00,316000000.0,0.000000e+00,-1.250000e+09,-2.156000e+09,0.000000e+00,0.0,7.160000e+08,151000000.0,0.0,-3.230000e+08,6.346090e+08,0.0,0.000000e+00,7.055000e+09,1.740000e+08,0.000000e+00
2,1,3,0.0,-5.713200e+07,783000.0,0.000000e+00,1.899600e+08,1.096714e+09,5.664200e+07,395000.0,0.000000e+00,0.0,172793000.0,6.258000e+07,6.911800e+07,-1442000.0,0.000000e+00,0.000000e+00,3.801700e+08,3.380860e+08
3,1,6,47000.0,-2.170000e+08,0.0,1.673000e+09,3.179000e+09,9.952000e+09,2.127000e+09,-41000000.0,5.000000e+08,0.0,219000000.0,5.080000e+08,1.767807e+09,-6000000.0,0.000000e+00,0.000000e+00,7.718000e+09,3.602000e+09
4,2,8,7600.0,-9.100000e+06,0.0,0.000000e+00,4.960000e+07,2.428000e+08,9.700000e+06,0.0,8.400000e+06,28900000.0,455600000.0,1.670000e+07,1.920000e+07,-100000.0,1.233000e+08,0.000000e+00,4.178000e+08,2.470000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,1,571,7100.0,-9.711000e+06,2064000.0,3.452200e+07,3.846500e+07,1.283150e+08,3.210600e+07,-1553000.0,0.000000e+00,0.0,22580000.0,6.551000e+06,6.230900e+07,0.0,5.736100e+07,3.561750e+08,1.505260e+08,1.565830e+08
265,1,573,22514.0,-1.730000e+08,0.0,0.000000e+00,-5.890000e+08,-1.830000e+08,0.000000e+00,5000000.0,1.590000e+08,0.0,448000000.0,-5.000000e+06,1.753270e+08,0.0,6.200000e+07,4.970000e+08,-3.930000e+08,6.000000e+06
266,1,574,0.0,-3.629000e+09,0.0,0.000000e+00,-2.007000e+10,-1.659400e+10,4.599328e+10,112000000.0,7.379000e+09,0.0,0.0,-6.010000e+09,4.270213e+09,-100000000.0,2.736000e+09,0.000000e+00,3.760000e+08,1.816500e+10
267,1,576,5100.0,-1.320000e+07,0.0,0.000000e+00,-9.540000e+07,1.761000e+08,7.890000e+07,5100000.0,0.000000e+00,0.0,56300000.0,-2.400000e+07,2.187000e+08,0.0,2.791000e+08,0.000000e+00,1.619000e+08,4.392000e+08


In [29]:
print(postcovid_features)
postcovid_df[postcovid_features['Features']]


                             Features   Coefficient
0                   Number of Layoffs  5.438326e-02
1                   industry_labelled  6.643478e-04
2                      employee_count  4.274054e-06
3                  capitalExpenditure  8.818919e-10
4                   commonStockIssued  7.899566e-10
5      ResearchAndDevelopmentExpenses  6.368232e-10
6                           netIncome  5.196202e-10
7                         grossProfit  4.266340e-10
8         depreciationAndAmortization  3.264021e-10
9          effectOfForexChangesOnCash  3.228086e-10
10                      otherExpenses  2.354769e-10
11        SellingAndMarketingExpenses  1.663071e-10
12      inventory_cash-flow-statement  1.558306e-10
13                   incomeTaxExpense  1.503780e-10
14              weightedAverageShsOut  1.498765e-10
15             commonStockRepurchased  8.939251e-11
16   GeneralAndAdministrativeExpenses  7.370204e-11
17          deferredRevenueNonCurrent  6.703442e-11
18          

,Number of Layoffs,industry_labelled,employee_count,capitalExpenditure,commonStockIssued,ResearchAndDevelopmentExpenses,netIncome,grossProfit,depreciationAndAmortization,effectOfForexChangesOnCash,otherExpenses,SellingAndMarketingExpenses,inventory_cash-flow-statement,incomeTaxExpense,weightedAverageShsOut,commonStockRepurchased,GeneralAndAdministrativeExpenses,deferredRevenueNonCurrent,freeCashFlow,deferrredTaxLiabilitiesNonCurrent
0,1,2,118033.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
1,1,6,47000.0,-1.620000e+08,0.0,1.497000e+09,4.490000e+09,9.486000e+09,2.053000e+09,7000000.0,0.000000e+00,0.0,-3.850000e+08,4.360000e+08,1.771000e+09,-1.470000e+09,0.0,0.0,4.746000e+09,2.831000e+09
2,1,6,47000.0,-1.770000e+08,0.0,1.614000e+09,3.949000e+09,9.790000e+09,2.205000e+09,-59000000.0,2.290000e+08,0.0,-1.700000e+07,4.480000e+08,1.771000e+09,-4.000000e+06,0.0,0.0,7.425000e+09,1.972000e+09
3,1,9,114000.0,-6.600000e+06,0.0,0.000000e+00,7.460000e+07,0.000000e+00,2.230000e+07,1900000.0,1.260200e+09,0.0,0.000000e+00,2.720000e+07,6.720000e+07,0.000000e+00,0.0,0.0,3.870000e+07,3.700000e+06
4,1,9,114000.0,-1.300000e+07,0.0,0.000000e+00,4.890000e+07,2.011100e+09,3.000000e+07,-1000000.0,1.593400e+09,0.0,0.000000e+00,1.490000e+07,6.640000e+07,-2.300000e+07,0.0,0.0,1.041000e+08,8.970000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2,562,48250.0,-5.500000e+08,0.0,0.000000e+00,5.870000e+08,1.885000e+09,5.080000e+08,0.0,5.220000e+08,0.0,0.000000e+00,1.890000e+08,4.144000e+08,-2.700000e+08,473000000.0,0.0,4.970000e+08,1.632000e+09
100,1,563,2200000.0,-3.539000e+09,0.0,0.000000e+00,2.054000e+09,3.472200e+10,2.680000e+09,49000000.0,0.000000e+00,0.0,-4.699000e+09,7.980000e+08,2.769000e+09,-2.408000e+09,0.0,0.0,-7.297000e+09,1.394300e+10
101,2,563,2200000.0,-3.953000e+09,0.0,0.000000e+00,5.149000e+09,3.702100e+10,2.699000e+09,-149000000.0,0.000000e+00,0.0,9.690000e+08,1.497000e+09,2.736000e+09,-3.339000e+09,0.0,0.0,9.045000e+09,1.409200e+10
102,1,566,49300.0,-1.810000e+08,0.0,0.000000e+00,3.990000e+07,1.003700e+09,3.736000e+08,14300000.0,8.810000e+07,0.0,-1.590000e+07,1.800000e+06,2.634000e+08,-2.193000e+08,0.0,0.0,2.089000e+08,2.850900e+09
